In [8]:
def acsu(n):
    return sum([2 for i in range(2, int(n**0.5)+1) if n%i ==0]) + 2 - (int(n**0.5) == n**0.5)*1

def solution(number, limit, power):
    return [min([acsu(i+1), power]) for i in range(number)]

solution(5,3,2)

[1, 2, 2, 2, 2]

In [1]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
import joblib
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('../../data/elo.csv')

def custom_train_test_split(df, ratio=0.7, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

def custom_train_test_split_fold_5(df, ratio=0.2, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data, k = 0, 0
    user_ids =[[],[],[],[],[]]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            k += 1
            max_train_data_len += ratio*len(df)
        user_ids[k].append(user_id)

    return user_ids

FEATS = [
   'KnowledgeTag', 'month', 'hour', 'week', 'elapsed', 'elapsed_cate',
   'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2',
   'testId0', 'testId1', 'test0_mean', 'test0_std', 'test1_mean',
   'test1_std', 'tag_mean', 'tag_std', 'ass0_mean', 'ass0_std',
   'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std',
'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']

cate = ['KnowledgeTag', 'month', 'hour', 'week', 'elapsed_cate', 'testId0', 'testId1',
       'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2',
        'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']

for i in cate:
    df[i] = df[i].astype('category')

In [23]:
# train, test = custom_train_test_split(df)

# y_train = train['answerCode']
# train = train.drop(['answerCode'], axis=1)

# y_test = test['answerCode']
# test = test.drop(['answerCode'], axis=1)
train, test, train_y, y_test = train_test_split(df[FEATS], df['answerCode'], test_size=0.2, shuffle=True, random_state=42)

In [24]:
lgb_train = lgb.Dataset(train[FEATS], train_y, categorical_feature=cate)
lgb_test = lgb.Dataset(test[FEATS], y_test, categorical_feature=cate)

In [25]:
params = {'objective': 'binary', 
          'metric': 'auc',
         }

model = lgb.train(
    {'objective': 'binary', 
      'metric': 'auc'},
    lgb_train,
    valid_sets=[lgb_test],
    verbose_eval=100,
    num_boost_round=10000,
    early_stopping_rounds=1000,
)

preds = model.predict(test[FEATS])
acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_test, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1323032, number of negative: 697732
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.065062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15397
[LightGBM] [Info] Number of data points in the train set: 2020764, number of used features: 29


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654719 -> initscore=0.639846
[LightGBM] [Info] Start training from score 0.639846
Training until validation scores don't improve for 1000 rounds
[100]	valid_0's auc: 0.824755


KeyboardInterrupt: 

In [10]:
from tqdm import tqdm

user_id = custom_train_test_split_fold_5(df)

for i in tqdm(range(5)):
    u = []
    for j in range(5):
        if j != i:
            u += user_id[j]
        
    train = df[df['userID'].isin(u)]
    test = df[df['userID'].isin(u) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    
    y_train = train['answerCode']
    train = train.drop(['answerCode'], axis=1)

    y_test = test['answerCode']
    test = test.drop(['answerCode'], axis=1)
    
    lgb_train = lgb.Dataset(train[FEATS], train_y, categorical_feature=cate)
    lgb_test = lgb.Dataset(test[FEATS], y_test, categorical_feature=cate)
    
    model = lgb.train(
        {'objective': 'binary',
        'metric': 'auc'}, 
        lgb_train,
        valid_sets=[lgb_train, lgb_test],
        verbose_eval=2000,
        num_boost_round=20000,
        early_stopping_rounds=2000,
    )

    preds = model.predict(test[FEATS])
    acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_test, preds)

    print(f'VALID AUC : {auc} ACC : {acc}\n')
    
    joblib.dump(model, f'lgb_{i}.pkl')

  0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1324846, number of negative: 696210
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.101120 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2617
[LightGBM] [Info] Number of data points in the train set: 2021056, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.655522 -> initscore=0.643400
[LightGBM] [Info] Start training from score 0.643400
Training until validation scores don't improve for 2000 rounds
[2000]	training's binary_logloss: 0.488578	valid_1's binary_logloss: 0.549361
[4000]	training's binary_logloss: 0.468831	valid_1's binary_logloss: 0.543875
Early stopping, best iteration is:
[3947]	training's binary_logloss: 0.469215	valid_1's binary_logloss: 0.543602


 20%|██        | 1/5 [14:25<57:42, 865.75s/it]

VALID AUC : 0.797653390980994 ACC : 0.7242547425474255



/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

 60%|██████    | 3/5 [26:28<20:06, 603.41s/it]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1321992, number of negative: 698756
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027068 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2617
[LightGBM] [Info] Number of data points in the train set: 2020748, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654209 -> initscore=0.637593
[LightGBM] [Info] Start training from score 0.637593
Training until validation scores don't improve for 2000 rounds
[2000]	training's binary_logloss: 0.489929	valid_1's binary_logloss: 0.546025
[4000]	training's binary_logloss: 0.470171	valid_1's binary_logloss: 0.542128
[6000]	training's binary_logloss: 0.454594	valid_1's binary_logloss: 0.540959
Early stopping, best iteration is:
[5641]	training's binary_logloss: 0.457188	valid_1's binary_logloss: 0.539906
VALID AUC : 0.802077230359

 80%|████████  | 4/5 [31:56<08:40, 520.79s/it]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1325260, number of negative: 695474
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2617
[LightGBM] [Info] Number of data points in the train set: 2020734, number of used features: 24
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.655831 -> initscore=0.644770
[LightGBM] [Info] Start training from score 0.644770
Training until validation scores don't improve for 2000 rounds
[2000]	training's binary_logloss: 0.490256	valid_1's binary_logloss: 0.561497
[4000]	training's binary_logloss: 0.470034	valid_1's binary_logloss: 0.553987
[6000]	training's binary_logloss: 0.454366	valid_1's binary_logloss: 0.548985
[8000]	training's binary_logloss: 0.441254	valid_1's binary_logloss: 0.548278
[10000]	training's binary_logloss: 0.429327	valid_1's binary_

100%|██████████| 5/5 [39:48<00:00, 477.68s/it]


In [60]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import StratifiedKFold

str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

X, y = df[FEATS], df['answerCode']

for i, (train_index, test_index) in tqdm(enumerate(str_kf.split(X, y))):
    train, test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    lgb_train = lgb.Dataset(train[FEATS], y_train, categorical_feature=cate)
    lgb_test = lgb.Dataset(test[FEATS], y_test, categorical_feature=cate)
    
    model = lgb.train(
        {'objective': 'binary',
        'metric': 'auc'}, 
        lgb_train,
        valid_sets=[lgb_train, lgb_test],
        verbose_eval=250,
        num_boost_round=10000,
#        early_stopping_rounds=500,
    )

    preds = model.predict(test[FEATS])
    acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_test, preds)

    print(f'VALID AUC : {auc} ACC : {acc}\n')
    
    joblib.dump(model, f'lgb_{i}.pkl')

0it [00:00, ?it/s]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322870, number of negative: 697894
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034159 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15147
[LightGBM] [Info] Number of data points in the train set: 2020764, number of used features: 28


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654639 -> initscore=0.639492
[LightGBM] [Info] Start training from score 0.639492
Training until validation scores don't improve for 500 rounds
[250]	training's auc: 0.828113	valid_1's auc: 0.81014
[500]	training's auc: 0.838476	valid_1's auc: 0.810736
[750]	training's auc: 0.844235	valid_1's auc: 0.810745
[1000]	training's auc: 0.849221	valid_1's auc: 0.810701
[1250]	training's auc: 0.853314	valid_1's auc: 0.810606
Early stopping, best iteration is:
[830]	training's auc: 0.84589	valid_1's auc: 0.810799
VALID AUC : 0.8107994139117203 ACC : 0.7676269616304296



1it [13:18, 798.24s/it]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


...

4it [39:42, 670.38s/it]/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 1322871, number of negative: 697894
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.075129 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15158
[LightGBM] [Info] Number of data points in the train set: 2020765, number of used features: 28


/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.654639 -> initscore=0.639492
[LightGBM] [Info] Start training from score 0.639492
Training until validation scores don't improve for 500 rounds
[250]	training's auc: 0.827844	valid_1's auc: 0.810845
[500]	training's auc: 0.838544	valid_1's auc: 0.811741
[750]	training's auc: 0.845421	valid_1's auc: 0.811813
[1000]	training's auc: 0.850003	valid_1's auc: 0.811652
Early stopping, best iteration is:
[680]	training's auc: 0.843612	valid_1's auc: 0.811861
VALID AUC : 0.8118611706677159 ACC : 0.7686261235849411



5it [1:02:13, 746.78s/it]


In [61]:
sub = pd.read_csv('../../data/infer.csv')
#sub = test[test['answerCode'] == -1]

p = []
for i in cate:
    sub[i] = sub[i].astype('category')
    
for i in tqdm(range(5)):
    load_model = joblib.load(f'lgb_{i}.pkl')
    preds = load_model.predict(sub[FEATS])
    p.append(preds)

s = pd.read_csv('output/submission.csv')
m = (p[0] + p[1] + p[2] + p[3] + p[4])/5
s['prediction'] = m

s.to_csv('output/submission_lgbm.csv', index=False)

100%|██████████| 5/5 [00:00<00:00,  8.76it/s]


In [46]:
#train = pd.read_csv('../../data/elo.csv')
#train.columns
q = pd.DataFrame([])
q['as0_as1'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID0), list(df.assessmentItemID1))])
q['as0_as2'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID0), list(df.assessmentItemID2))])
q['as1_as2'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID1), list(df.assessmentItemID2))])
q['week_hour'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.week), list(df.hour))])


sub['week_hour'] =  [str(i)+'_'+str(j) for i, j in zip(list(sub.week), list(sub.hour))]
sub['as0_as1'] =  [str(i)+'_'+str(j) for i, j in zip(list(sub.assessmentItemID0), list(sub.assessmentItemID1))]
sub['as0_as2'] =  [str(i)+'_'+str(j) for i, j in zip(list(sub.assessmentItemID0), list(sub.assessmentItemID2))]
sub['as1_as2'] =  [str(i)+'_'+str(j) for i, j in zip(list(sub.assessmentItemID1), list(sub.assessmentItemID2))]


cate = ['as0_as1', 'as0_as2', 'as1_as2', 'week_hour']
label = [as0_as1, as0_as2, as1_as2, week_hour]
for index, col in enumerate(cate):
    cate2label = {j:i for i,j in enumerate(q[col].unique())}
    sub[col] = sub[col].map(cate2label)
    
sub[FEATS[15:]]

,tag_mean,tag_std,ass0_mean,ass0_std,ass1_mean,ass1_std,ass2_mean,ass2_std,elo_prob,as0_as1,as0_as2,as1_as2,assessmentItemID,week_hour
0,0.559692,0.496500,0.659086,0.474017,0.654069,0.475687,0.455291,0.498000,0.473301,422,67,1273,A050133008,64
1,0.547059,0.497878,0.521662,0.499531,0.752487,0.431590,0.455291,0.498000,0.569538,515,76,1471,A070146008,22
2,0.494333,0.500051,0.521662,0.499531,0.648133,0.477569,0.455291,0.498000,0.191112,984,76,1330,A070111008,61
3,0.420000,0.493849,0.454675,0.497943,0.648571,0.477437,0.556916,0.496751,0.394117,1202,37,1234,A090064006,111
4,0.609474,0.487920,0.712236,0.452722,0.646154,0.478178,0.515859,0.499750,0.345771,911,5,1188,A060135007,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0.701429,0.457740,0.680253,0.466379,0.673308,0.469021,0.599453,0.490010,0.101355,951,31,1251,A040122005,17
740,0.824583,0.380363,0.702195,0.457294,0.648133,0.477569,0.599453,0.490010,0.761471,1107,52,971,A030111005,73
741,0.823881,0.380979,0.659086,0.474017,0.772090,0.419516,0.663597,0.472479,0.782659,1462,55,1384,A050193004,22
742,0.823881,0.380979,0.659086,0.474017,0.772090,0.419516,0.663597,0.472479,0.597028,1462,55,1384,A050193004,90


In [62]:
a = pd.read_csv('output/output (3).csv')
b = pd.read_csv('output/submission_lgbm.csv')

c = (a['prediction'] + b['prediction'])/2
b['prediction'] = c
b.to_csv('output/h.csv', index=False)